In [18]:
import pandas as pd
from PIL import Image, ImageDraw, ImageOps
import praw
from datetime import datetime, timedelta 
import math
import os
from imgurpython import ImgurClient
import time

In [2]:
#reddit account setup
reddit = praw.Reddit(client_id='sBuZonrzHbbn0a-GQ9U5QQ',
    client_secret='4yGT8iIBmvAslGY_QVYDV4PurHWioQ',
    username='Bishoplover69',
    password='23hkpOE3vL2@8zKIkNWl&35JXFY@vsF26U26TMhxP8o#arv5ihp#RZbFG#ako&#3M2bzv',
    user_agent='BishopMosaicApp/0.0.1 (by u/Bishoplover69)')
print(reddit.user.me())

Bishoplover69


In [16]:
#imgur acc
imgur_client_id = '47f2575afbb54a8'
imgur_client_secret = 'a2d3a7e6761adf37d7060a7b76b33cf88e6293a8'

In [19]:
#imgur bild upload
cwd = os.getcwd()
imgur_client = ImgurClient(imgur_client_id, imgur_client_secret)
image_path = os.path.join(cwd, "leer.png")
uploaded_image = imgur_client.upload_from_path(image_path, config=None, anon=True)

image_url = uploaded_image['link']
print(image_url)


https://i.imgur.com/zNK0VFt.png


In [11]:
#reddit post
#funktion für likes geben schreiben
subreddit_name = "AnarchyChess"
likes = 1420

#posttietel rein machen
post_title = "lol"
datum_post = datetime.now("%Y-%m-%d %H:%M:%S")
day_after_post = datum_post + timedelta(hours=24)
day_after_post = day_after_post.strftime("%Y-%m-%d %H:%M:%S")

#subreddit = reddit.subreddit(subreddit_name)
#subreddit.submit(post_title, url=image_url)


In [12]:
cwd = os.getcwd()
csv_file = os.path.join(cwd, "aja.csv")

In [13]:
#csv magic
if os.path.isfile(csv_file):
    df = pd.read_csv(csv_file)
else:
    df = pd.DataFrame()

likes_sum = int(df['likes'].sum()) if 'likes' in df.columns else 0
likes_added = likes + int(likes_sum)

new_row = pd.DataFrame({'post_date': [datum_post], 'post_title': [post_title], 'likes': [likes], 'likes_added': [likes_added]})
df = pd.concat([df, new_row], ignore_index=True)


df.to_csv(csv_file, index=False)

In [14]:
#imagetest
anz_bishops = 65536
images_to_add = likes
images_to_process = min(anz_bishops, likes_sum + images_to_add)

img_bishop = Image.open("BISHOP.png")
canvas = Image.new('RGB', (8192,8192))
target = Image.open("Bishop_full.png")

canvas_size = 8192
b_size = 512

num_cols = int(math.ceil(math.sqrt(anz_bishops)))
num_rows = int(math.ceil(anz_bishops / num_cols))

b_size = min(canvas_size // num_cols, canvas_size // num_rows)
spacing = (canvas_size - b_size * num_cols) // (num_cols + 1)


#target image to grid
cell_width = canvas_size // num_cols
cell_height = canvas_size // num_rows

grid = [[0] * num_cols for _ in range(num_rows)]
id_counter = 1
for row in range(num_rows):
    for col in range(num_cols):
        grid[row][col] = id_counter
        id_counter += 1

def get_average_color(image, row, col, cell_width, cell_height):
    x1 = col*cell_width
    y1 = row*cell_height
    x2 = x1+cell_width
    y2 = y1 + cell_height

    r_total, g_total, b_total = 0,0,0
    pixel_count = 0
    for x in range (x1,x2):
        for y in range (y1,y2):
            r,g,b = image.getpixel((x,y))
            r_total += r
            g_total += g
            b_total += b
            pixel_count += 1
    
    r_avg = r_total // pixel_count
    g_avg = g_total // pixel_count
    b_avg = b_total // pixel_count

    return r_avg, g_avg, b_avg
def colorize_image(image, target_color):
    grayscale_img = ImageOps.grayscale(image)

    colored_img = ImageOps.colorize(grayscale_img, 'black', target_color)
    return colored_img


# Step 5: Replace grid cells with small images
for row in range(num_rows):
    for col in range(num_cols):
        img_index = row * num_cols + col
        if img_index < images_to_process:
            # Resize and paste image
            
            average_color = get_average_color(target, row, col, cell_width, cell_height)
                
            colored_img_bishop = colorize_image(img_bishop, average_color)

            resized_colored_img_bishop = colored_img_bishop.resize((b_size, b_size), resample=Image.LANCZOS)
            x = spacing + col * (b_size + spacing)
            y = spacing + row * (b_size + spacing)
            canvas.paste(resized_colored_img_bishop, (x, y))
# Step 6: Merge the small images
draw = ImageDraw.Draw(canvas)
canvas.save("test.png")